In [332]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import re
import datetime
import json
import os

In [293]:
s = requests.Session()
params = {"login": "johannp@esplus.de", "password": "schafkopfbruder"}

In [294]:
r = s.get("https://www.sauspiel.de/login")
html = r.text
soup = BeautifulSoup(html, "html.parser")
auth_token = soup.find("form").find_all("input")[1]["value"]
r = s.post("https://www.sauspiel.de/login", params={'login': 'johannp@esplus.de', 'password': 'schafkopfbruder',
                                                    'authenticity_token': auth_token})
games = s.get("https://www.sauspiel.de/spiele/35000001")
html = games.text
bs = BeautifulSoup(html, "html.parser")
print(games.text)

<!DOCTYPE html>
<html xmlns:fb="http://www.facebook.com/2008/fbml" lang="de" class="no-js">
  <head>
    <title>Sauspiel - Schafkopf für Freunde</title>

    <meta name="csrf-param" content="authenticity_token" />
<meta name="csrf-token" content="WPFYByujlPpwGyBvVPbIOxbk3jOBVrAjG2GKoLL7XNo=" />
    <meta http-equiv="Content-Type" content="text/html; charset=UTF-8">
    <meta http-equiv="content-language" content="de">
    <meta name="author" content="Sauspiel GmbH">
    <meta name="description" content="Sauspiel ist die größte Schafkopf-Community. Hier kann um Punkte und um Geld Schafkopf gespielt werden.">
    <meta name="keywords" content="Schafkopf, Schafkopf online, Internet-Schafkopf, Online-Schafkopf, Internet-Schafkopf, Sauspiel Community">
    <meta http-equiv="imagetoolbar" content="no">
    <meta name="viewport" content="width=device-width, initial-scale=1, user-scalable=no, shrink-to-fit=no">
    <meta name="application-name" content="Sauspiel">
    <meta name="msapplication

In [255]:
game_data = {
    "game_number": 0,
    "date": None,
    "hour": "",
    "money_won": 0.0, #
    "player_name": "", #
    "C0": "", #
    "C1": "",
    "C2": "",
    "C3": "",
    "C4": "",
    "C5": "",
    "C6": "",
    "C7": "", #
    "modus": "", #
    "position": 0, #
    "role": "", ## player role ["Spieler", "Mitspieler", "Gegenspieler"]
    "won": "", #
    "points": 0, #
    "laufende": 0,
    "kurze": 0,
    "geier": 0, #
    "f_wenz": 0, #
    "ramsch": 0, #
}

all_cards = np.array([
    "S7", "S8", "S9", "SX", "SK", "SA", "SU", "SO",
    "H7", "H8", "H9", "HX", "HK", "HA", "HU", "HO",
    "G7", "G8", "G9", "GX", "GK", "GA", "GU", "GO",
    "E7", "E8", "E9", "EX", "EK", "EA", "EU", "EO",
])

In [ ]:
player_name = bs.findAll("a", {"class": "avatar"})[0].get("data-username")
cards = [np.intersect1d(np.array(r.get("class")), all_cards)[0]
     for r in bs.find("div", {"id": f"{player_name}_Karten"}).findAll("span", {"class": "card-image"})]
rules = [e["title"] for e in bs.find("th", string="Sonderregeln").find_next_sibling().findChildren()]
date = bs.find("h2", {"class": "title-supertext"}).text.strip()
date = re.split(", | |\.", date)[-6:-2]
date.remove("")
date = "-".join(date)
hour = bs.find("h2", {"class": "title-supertext"}).text.strip()
hour = re.split(", | |\.", hour)[-1]

game_data = {
                "game_number": f"{game_number}.{position}",
                "date": date,
                "hour": hour,
                "money_won": float(bs.findAll("a", {"class": "avatar"})[position]\
                    .parent.find_next_sibling()\
                    .get_text(separator="", strip=True).split("€")[1].replace(",", ".")), #

                "player_name": bs.findAll("a", {"class": "avatar"})[position]\
                    .get("data-username"), #

                "cards": cards,
                "modus": self.get_game_mode(bs),
                "position": position,
                "role": bs.findAll("a", {"class": "avatar"})[position]\
                    .parent.find_next_sibling()\
                    .get_text(separator="", strip=True)\
                    .split("€")[0],

                "won": self.format_won(bs.find("th", string="Spielausgang").find_next_sibling().text.strip().split(" ")[0]), #
                "points": [int(r) for r in bs.find("th", string="Spielausgang").find_next_sibling().text.strip().split(" ")
                      if r.isdigit()][0],

                "laufende": int(bs.find("th", string="Laufende").find_next_sibling().text.strip()),
                "kurze": 1 if "Kurze Karte" in rules else 0,
                "geier": 1 if "Geier" in rules else 0, #
                "f_wenz": 1 if "Farbwenz" in rules else 0, #
                "ramsch": 1 if "Ramsch" in rules else 0
            }

In [243]:

modus = bs.find("h1").text.split(" ")[0]
won = bs.find("th", string="Spielausgang").find_next_sibling().text.strip().split(" ")[0]
points = [int(r) for r in bs.find("th", string="Spielausgang").find_next_sibling().text.strip().split(" ")
          if r.isdigit()][0]
rules = [e["title"] for e in bs.find("th", string="Sonderregeln").find_next_sibling().findChildren()]
geier = 1 if "Geier" in rules else 0
kurze = 1 if "Kurze Karte" in rules else 0
f_wenz = 1 if "Farbwenz" in rules else 0
position = 0 #TODO this will be declared in a for-loop
player_name = bs.findAll("a", {"class": "avatar"})[position].get("data-username")
role = bs.findAll("a", {"class": "avatar"})[position].parent.find_next_sibling().get_text(separator="", strip=True).split("€")[0]
money_won = float(bs.findAll("a", {"class": "avatar"})[position]\
    .parent.find_next_sibling()\
    .get_text(separator="", strip=True).split("€")[1].replace(",", "."))
cards = [np.intersect1d(np.array(r.get("class")), all_cards)[0]
         for r in bs.find("div", {"id": f"{player_name}_Karten"}).findAll("span", {"class": "card-image"})]
date = bs.find("h2", {"class": "title-supertext"}).text.strip()
date = re.split(", | |\.", date)[-6:-2]
date.remove("")
date = "-".join(date)
hour = bs.find("h2", {"class": "title-supertext"}).text.strip()
hour = re.split(", | |\.", hour)[-1]
laufende = int(bs.find("th", string="Laufende").find_next_sibling().text.strip())
print(laufende)

0


In [258]:
x = {"a"}
x.

{'a'}

In [268]:
def get_game_mode(bs):
    asse = {"Alte", "Hundsgefickte", "Blaue"}
    header = bs.find("h1").text.split(" ")
    if "Sauspiel" in header:
        ass = set(header).intersection(asse)
        return f"Sauspiel-{ass}"
    return bs.find("h1").text.split(" ")[0]

def format_won(won):
    return 1 if won == "Gewonnen" else 0

def check_game(bs_all):
    return len(bs_all.findAll("div", {"class": "card-rows"})) != 0

In [269]:
bs.findAll("div", {"class": "card-rows"})

[]

In [271]:
check_game(bs)

False

In [469]:
all_cards = np.array([
    "S7", "S8", "S9", "SX", "SK", "SA", "SU", "SO",
    "H7", "H8", "H9", "HX", "HK", "HA", "HU", "HO",
    "G7", "G8", "G9", "GX", "GK", "GA", "GU", "GO",
    "E7", "E8", "E9", "EX", "EK", "EA", "EU", "EO",
])


class SchafkopfScraper:
    def __init__(self, search_start = 35_000_000, max_games = 10):
        self.games = []
        self.max_games = max_games
        self.games_searched = 0
        self.games_found = 0
        self.private_games = 0
        self.search_start = search_start
        self.max_games = self.max_games
        self.all_cards = np.array([
        "S7", "S8", "S9", "SX", "SK", "SA", "SU", "SO",
        "H7", "H8", "H9", "HX", "HK", "HA", "HU", "HO",
        "G7", "G8", "G9", "GX", "GK", "GA", "GU", "GO",
        "E7", "E8", "E9", "EX", "EK", "EA", "EU", "EO",
    ])

    def start_search(self):
        s = requests.Session()
        r = s.get("https://www.sauspiel.de/login")
        html = r.text
        soup = BeautifulSoup(html, "html.parser")
        auth_token = soup.find("form").find_all("input")[1]["value"]
        r = s.post("https://www.sauspiel.de/login", params={'login': 'johannp@esplus.de', 'password': 'schafkopfbruder',
                                                            'authenticity_token': auth_token})

        current_game = self.search_start
        while self.games_searched < self.max_games:
            game = s.get(f"https://www.sauspiel.de/spiele/{current_game}")
            html = game.text
            bs = BeautifulSoup(html, "html.parser")
            if self.check_game(bs):
                print("###############################")
                print(f"### game : {current_game} ###")
                res = self.get_game_batch(bs, current_game)
                self.games.extend(res)
                self.games_found += 1
                print("Something went wrong")
            else:
                print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
                print(f"~~~ Skipping game: {current_game} ~~~")
                self.private_games += 1

            self.games_searched += 1
            current_game += 1
            print(f"games searched: {self.games_searched}\n"
                  f"games registerd: {self.games_found}\n"
                  f"private games: {self.private_games}")


        with open("../data/raw/games.json", "w+") as f:
            out = {"games": self.games}
            json.dump(out, f)
        print("done")

    def write_json(self, results_dir, new_data, file_name = "games.json"):
        file_path = f"{results_dir}/{file_name}"
        if not os.path.isdir(results_dir):
            os.makedirs(results_dir)
        with open(file_path, "w+") as file:
            if not os.path.isfile(results_dir):
                empty_json = {"games": []}
                json.dump(empty_json, file)
            file_data = json.loads(file)
            file_data["games:"].append(new_data)
            file.seek(0)
            json.dump(file_data, file, indent=4)


    def get_game_mode(self, bs):
        asse = {"Alte", "Hundsgfickte", "Blaue"}
        header = bs.find("h1").text.split(" ")
        if "Zamgworfen" in header:
            return "Zamgworfen"
        if "Sauspiel" in header:
            ass = set(header).intersection(asse)
            return f"Sauspiel-{ass.pop()}"
        return bs.find("h1").text.split(" ")[0]

    def format_won(self, won):
        return 1 if won == "Gewonnen" else 0

    def check_game(self, bs_all):
        played = self.get_game_mode(bs_all) != "Zamgworfen"
        public = len(bs_all.findAll("div", {"class": "card-rows"})) != 0
        return played and public

    def get_game_batch(self, bs, game_number):
        batch = []
        for position in range(4):
            player_name = bs.findAll("a", {"class": "avatar"})[position].get("data-username")
            cards = [np.intersect1d(np.array(r.get("class")), all_cards)[0]
                 for r in bs.find("div", {"id": f"{player_name}_Karten"}).findAll("span", {"class": "card-image"})]
            rules = [e["title"] for e in bs.find("th", string="Sonderregeln").find_next_sibling().findChildren()]
            date = bs.find("h2", {"class": "title-supertext"}).text.strip()
            date = re.split(", | |\.", date)[-6:-2]
            date.remove("")
            date = "-".join(date)
            hour = bs.find("h2", {"class": "title-supertext"}).text.strip()
            hour = re.split(", | |\.", hour)[-1]

            game_data = {
                "game_number": f"{game_number}.{position}",
                "date": date,
                "hour": hour,
                "money_won": float(bs.findAll("a", {"class": "avatar"})[position]\
                    .parent.find_next_sibling()\
                    .get_text(separator="", strip=True).split("€")[1].replace(",", ".")), #

                "player_name": player_name, #
                "cards": cards,
                "modus": self.get_game_mode(bs),
                "position": position,
                "role": bs.findAll("a", {"class": "avatar"})[position]\
                    .parent.find_next_sibling()\
                    .get_text(separator="", strip=True)\
                    .split("€")[0],

                "won": self.format_won(bs.find("th", string="Spielausgang").find_next_sibling().text.strip().split(" ")[0]), #
                "points": [int(r) for r in bs.find("th", string="Spielausgang").find_next_sibling().text.strip().split(" ")
                      if r.isdigit()][0],

                "laufende": int(bs.find("th", string="Laufende").find_next_sibling().text.strip()),
                "kurze": 1 if "Kurze Karte" in rules else 0,
                "geier": 1 if "Geier" in rules else 0, #
                "f_wenz": 1 if "Farbwenz" in rules else 0, #
                "ramsch": 1 if "Ramsch" in rules else 0
            }
            batch.append(game_data)
        return batch

In [470]:
searcher = SchafkopfScraper(max_games=10000)
searcher.start_search()

~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~ Skipping game: 35000000 ~~~
games searched: 1
games registerd: 0
private games: 1
###############################
### game : 35000001 ###
Something went wrong
games searched: 2
games registerd: 1
private games: 1
~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~ Skipping game: 35000002 ~~~
games searched: 3
games registerd: 1
private games: 2
~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~ Skipping game: 35000003 ~~~
games searched: 4
games registerd: 1
private games: 3
~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~ Skipping game: 35000004 ~~~
games searched: 5
games registerd: 1
private games: 4
~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~ Skipping game: 35000005 ~~~
games searched: 6
games registerd: 1
private games: 5
~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~ Skipping game: 35000006 ~~~
games searched: 7
games registerd: 1
private games: 6
###############################
### game : 35000007 ###
Something went wrong
games searched: 8
games registerd: 2
private games: 6
###############################
### game : 35000

AttributeError: 'NoneType' object has no attribute 'text'

In [373]:
r = {"games": []}
with open('data.json', 'w+') as f:
    json.dump(r, f)

In [410]:
new_data = {"a": 2, "b": 3}

In [443]:
empty_json = {"games": []}
with open("data.json", "w+", encoding="utf-8") as file:
    json.dump(empty_json, file)

In [444]:
with open("data.json", "w+", encoding="utf-8") as file:
    file_data = json.load(file)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [436]:
with open("data.json") as f:
    file_data = json.load(f)
    file_data["games"].append(new_data)
    print(file_data)
    file.seek(0)
    json.dump(file_data, f, indent=4)

{'games': ['{"a": 2, "b": 3}']}


ValueError: I/O operation on closed file.

In [135]:
s = "Gewonnen mit 104 Augen (Schneider)"
x = re.search("[0-9]", s)
x

<re.Match object; span=(13, 34), match='104 Augen (Schneider)'>

In [180]:
float("0,34".replace(",", "."))

0.34

In [300]:
x = {"a"}
x.pop()

'a'